In [1]:
## Preprocess each individual BEHAVIORAL data file to create the "State" column

In [ ]:
import pandas as pd
import os

# Path to the folder containing participant files
folder_path = ""

# Directory where you want to save the updated files (it is advised to create a new folder)
output_directory = ''

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Function to extract subject number from filename
def extract_subject_number(filename):
    # Extract digits from filename
    subject_number = ''.join(filter(str.isdigit, filename))
    return subject_number

# Function to process each CSV file
def process_csv_file(file_path):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    
    # Initialize column "State"
    df["State"] = "None"
    
    # Extract subject number from filename
    subject_number = extract_subject_number(os.path.basename(file_path))
    
    # Add subject number as a new column
    df.insert(0, "SubjectNumber", subject_number)
    
    # Organize dataset by difficulty and block number
    difficulties = df["difficulty"].unique()

    for d in difficulties:
        df_d = df.loc[df["difficulty"] == d]
        blocksNumbers = df_d["blockNumber"].unique()
        for b in blocksNumbers:
            df_b = df_d.loc[df_d["blockNumber"] == b].copy()  # Create a copy of the DataFrame
            df_b.reset_index(inplace=True)
            ## COLUMN STATE
            # Set first line of each block as Exploration
            df_b.at[0, "State"] = "T1"
            # Second line
            df_b.at[1, "State"] = "Exploitation" if df_b["accuracy"].loc[1:3].sum() == 3 else "Exploration"
            # Third line
            df_b.at[2, "State"] = "Exploitation" if df_b["accuracy"].loc[1:3].sum() == 3 or df_b["accuracy"].loc[2:4].sum() == 3 else "Exploration"
            # Fourth to eighth lines
            for i in range(3, 8):
                df_b.at[i, "State"] = "Exploitation" if (
                    df_b["accuracy"].loc[i-2:i].sum() == 3 or
                    df_b["accuracy"].loc[i-1:i+1].sum() == 3 or
                    df_b["accuracy"].loc[i:i+2].sum() == 3 or
                    df_b["accuracy"].loc[i-2:i+2].sum() == 4
                ) else "Exploration"
            # Ninth line
            df_b.at[8, "State"] = "Exploitation" if (
                df_b["accuracy"].loc[6:8].sum() == 3 or
                df_b["accuracy"].loc[7:9].sum() == 3 or
                df_b["accuracy"].loc[5:9].sum() == 4
            ) else "Exploration"
            # Tenth line
            df_b.at[9, "State"] = "Exploitation" if df_b["accuracy"].loc[7:9].sum() == 3 else "Exploration"

    # Return the processed DataFrame
    return df

# Iterate over each file in the specified folder
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        # Process the CSV file
        processed_data = process_csv_file(file_path)
        # Save the updated DataFrame to a new CSV in the output directory
        output_file = os.path.join(output_directory, filename)
        processed_data.to_csv(output_file, index=False)

print(f"Updated files have been saved to {output_directory}")


In [ ]:
## Transfer the "State" column to the pupillometry data files

In [ ]:
import pandas as pd
import os

# Paths
csv_folder_path = ""  # Path to the folder containing participant CSV files
excel_folder_path = ""  # Path to the folder containing participant Excel files
output_excel_folder = ""  # Path to save the updated Excel files

# Create the output directory if it doesn't exist
os.makedirs(output_excel_folder, exist_ok=True)

# Function to extract subject number from filename
def extract_subject_number(filename):
    subject_number = ''.join(filter(str.isdigit, filename))
    return subject_number

# Function to update Excel files with State data from CSV
def update_excel_file(csv_df, excel_file_path, output_path):
    # Load the Excel file
    excel_data = pd.read_excel(excel_file_path, sheet_name=None)  # Read all sheets
    updated_sheets = {}

    # Loop through each sheet in the Excel file
    for sheet_name, df in excel_data.items():
        if {"difficulty", "blockNumber", "trialNumber"}.issubset(df.columns):
            # Add the "State" column initialized with None
            df["State"] = None

            # Iterate through the rows of the CSV data
            for _, csv_row in csv_df.iterrows():
                # Get the specific combination of difficulty, blockNumber, and trialNumber
                difficulty = csv_row["difficulty"]
                block_number = csv_row["blockNumber"]
                trial_number = csv_row["trialNumber"]
                state = csv_row["State"]

                # Update all rows in the Excel sheet that match this combination
                matching_rows = (
                    (df["difficulty"] == difficulty) &
                    (df["blockNumber"] == block_number) &
                    (df["trialNumber"] == trial_number)
                )
                df.loc[matching_rows, "State"] = state

        # Add the updated sheet to the dictionary
        updated_sheets[sheet_name] = df

    # Save the updated Excel file
    with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
        for sheet_name, updated_df in updated_sheets.items():
            updated_df.to_excel(writer, index=False, sheet_name=sheet_name)

# Process each CSV file
for csv_filename in os.listdir(csv_folder_path):
    if csv_filename.endswith(".csv"):
        # Read the processed CSV file
        csv_path = os.path.join(csv_folder_path, csv_filename)
        csv_df = pd.read_csv(csv_path)

        # Extract subject number
        subject_number = extract_subject_number(csv_filename)

        # Find the corresponding Excel file
        excel_filename = f"subject_{subject_number}.xlsx"  # Adjust naming convention if needed
        excel_path = os.path.join(excel_folder_path, excel_filename)

        if os.path.exists(excel_path):
            # Define output path for the updated Excel file
            output_excel_path = os.path.join(output_excel_folder, excel_filename)

            # Update the Excel file
            update_excel_file(csv_df, excel_path, output_excel_path)

print(f"Updated Excel files have been saved to {output_excel_folder}")
